In [1]:
# 🐍 SECTION 1: Imports and Setup
import tensorflow as tf
from tensorflow.keras.applications import MobileNet, MobileNetV2, MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt

# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5  # 5 dog breeds
TEST_CSV = "test_labels.csv"
TEST_DIR = "test/"


In [6]:
# 📂 SECTION 2: Load Test Data
test_df = pd.read_csv("test.csv")
test_df['label'] = test_df['label'].astype(str)  # Ensure labels are string

test_gen = ImageDataGenerator(rescale=1./255)

test_data = test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory=TEST_DIR,
    x_col="filename",
    y_col="label",
    target_size=IMAGE_SIZE,
    class_mode="categorical",
    shuffle=False,
    batch_size=1
)


Found 206 validated image filenames belonging to 5 classes.


In [7]:
# 🧠 SECTION 3: Load Model With Custom Head (Frozen Base)
def load_mobilenet_variant(base_model_class, name):
    base_model = base_model_class(
        input_shape=(224, 224, 3),
        weights='imagenet',
        include_top=False
    )
    base_model.trainable = False  # Freeze backbone

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.3)(x)
    output = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output, name=name)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [8]:
# 🧪 SECTION 4: Evaluate All MobileNet Variants (No Training)
mobilenets = {
    "MobileNetV1": MobileNet,
    "MobileNetV2": MobileNetV2,
    "MobileNetV3": MobileNetV3Large
}

results = {}

for name, model_class in mobilenets.items():
    print(f"\n🔍 Evaluating {name} ...")
    model = load_mobilenet_variant(model_class, name)
    loss, acc = model.evaluate(test_data, verbose=1)
    results[name] = acc



🔍 Evaluating MobileNetV1 ...


c:\Users\anand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


206/206 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.3245 - loss: 1.9897

🔍 Evaluating MobileNetV2 ...
206/206 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.2694 - loss: 1.9231

🔍 Evaluating MobileNetV3 ...
206/206 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.1890 - loss: 1.6515


In [9]:
# 📊 SECTION 5: Display Results
print("\n📈 Test Accuracy Comparison (Random Head, No Training):")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")



📈 Test Accuracy Comparison (Random Head, No Training):
MobileNetV1: 0.2864
MobileNetV2: 0.2184
MobileNetV3: 0.2233
